In [1]:
import tensorflow as tf
import numpy as np

In [2]:
tf.reset_default_graph()

In [3]:
# dataset
char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz'] # S start, E end, P padding
num_dic = {n: i for i, n in enumerate(char_arr)}
seq_data = [['man', 'women'], ['black', 'white'], ['king', 'queen'], ['girl', 'boy'], ['up', 'down'], ['high', 'low']]

In [4]:
# parameter
n_step = 5
n_hidden = 128
n_classes = len(num_dic)

In [5]:
def make_batch(seq_data):
    input_batch, output_batch, target_batch = [], [], []
    # input_batch : (batch_size, time_step, input_size(one-hot))
    # output_batch : (batch_size, time_step + 1, output_size(one-hot))
    # target_batch : (batch_size, time_step + 1)
    
    for seq in seq_data:
        for i in range(2):
            seq[i] = seq[i] + 'P' * (n_step - len(seq[i]))
        
        input = [num_dic[n] for n in seq[0]]
        output = [num_dic[n] for n in ('S' + seq[1])]
        target = [num_dic[n] for n in (seq[1] + 'E')]
        
        input_batch.append(np.eye(n_classes)[input])
        output_batch.append(np.eye(n_classes)[output])
        
        target_batch.append(target)
        
    return input_batch, output_batch, target_batch

## Model

In [6]:
enc_input = tf.placeholder(tf.float32, [None, None, n_classes])
dec_input = tf.placeholder(tf.float32, [None, None, n_classes])
target = tf.placeholder(tf.int32, [None, None])

In [7]:
with tf.variable_scope('encoder'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
    _, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype=tf.float32)

with tf.variable_scope('decoder'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    outputs, _ = tf.nn.dynamic_rnn(dec_cell, dec_input, initial_state=enc_states, dtype=tf.float32)

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
model = tf.layers.dense(outputs, n_classes, activation=None)

Instructions for updating:
Use keras.layers.dense instead.


In [9]:
# loss
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=target))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

## Train

In [10]:
# train
sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch, output_batch, target_batch = make_batch(seq_data)

In [11]:
for epoch in range(5000):
    _, loss = sess.run([optimizer, cost], feed_dict={enc_input: input_batch, dec_input: output_batch, target: target_batch})
    if (epoch + 1) % 1000 == 0:
        print('Epoch: %04d, cost: %.4f' % (epoch+1, loss))

Epoch: 1000, cost: 0.0010
Epoch: 2000, cost: 0.0002
Epoch: 3000, cost: 0.0001
Epoch: 4000, cost: 0.0000
Epoch: 5000, cost: 0.0000


## TEST

In [ ]:
tf.argmax()

In [24]:
def translate(word):
    seq_data = [word, 'P' * len(word)]
    
    input_batch, output_batch, _ = make_batch([seq_data])
    prediction = tf.argmax(model, 2) # model : (batch_size, time_step, input_size)
    
    result = sess.run(prediction, {enc_input: input_batch, dec_input: output_batch})
    
    decoded = [char_arr[i] for i in result[0]]
    end = decoded.index('E')
    translated = ''.join(decoded[:end])
    
    return translated.replace('P', '')

In [25]:
print('man -> %s' % (translate('man')))
print('mans -> %s' % (translate('mans')))
print('king -> %s' % (translate('king')))
print('black -> %s' % (translate('black')))
print('upp -> %s' % (translate('upp')))

man -> women
mans -> women
king -> queen
black -> white
upp -> down
